In [16]:
!pip install vaderSentiment

In [17]:
!pip install paralleldots

In [18]:
import html
import pandas as pd
from html.parser import HTMLParser
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import requests
import numpy as np

In [19]:
pd.set_option('display.max_colwidth', -1)

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/CVanchieri/DataSets/master/HackerNews/2007TopCommentorsComments.csv')
print(df.shape)
df.head(10)

(35029, 7)


,parent,id,time,timestamp,by,text,score
0,39604,39629,1186355469,2007-08-05 23:11:09 UTC,Jd,"I'm in the 'Well duh' category too, but I still think Apple is over-rated. Too much enthusiasm for Jobs &#38; co. will do no one any good.",NaN
1,39057,39157,1186178497,2007-08-03 22:01:37 UTC,Jd,perhaps clearer if I wrote 'profitability and the quality of the content',NaN
2,57130,57242,1190262050,2007-09-20 04:20:50 UTC,Jd,Does anything like this exist that you are aware of?,NaN
3,32291,32294,1183475440,2007-07-03 15:10:40 UTC,Jd,"Depends on where you go for college. The ivies I am familiar with separate the applications and financial aid process the issue of 'leniency' is not there (except when there are very large sums of money involved).<p>As for private schools, I think you may be confused. Many of the leading private boarding schools are <i>much</i> better than almost any public schools in the nation. Counseling programs vary with the school. Schools which frequently send kids to the ivies (or ivy-equiv) tend to have lots of experience with the process. My public school had only a small amount. Some have none (and don't even know when one of their students has beaten the odds and made their way into such a school).<p>",NaN
4,57812,57919,1190380082,2007-09-21 13:08:02 UTC,Jd,"Speculating on a causal principle implies that there is causality in the universe. Consequently, order and the possiblity of some degree of understanding are also implied, although the translation from speculation to empirical confirmation is not.",NaN
5,46389,46532,1188030337,2007-08-25 08:25:37 UTC,Jd,"This is a simple case of specialization of labor. While a hacker may be able to hear the voice of the customer, it probably makes sense to have some folks that do more hacking and some more customer hearing. If you have two co-founders, it makes sense to have one on each side.",NaN
6,39609,39627,1186355326,2007-08-05 23:08:46 UTC,Jd,"'Profit' is usually defined as pecuniary benefit. PG has stated elsewhere that he is not particularly concerned about this. <p>In my charitable estimation, PG's ""Not a charity"" statement means that YC is not in the business of giving out financial handouts. It does not mean that PG's first or only motive is pecuniary benefit.<p>We live in a culture defined by the worship of the Almighty dollar. If PG is at least hinting in a different direction, I'd say that's a good thing.",NaN
7,55442,55479,1189994744,2007-09-17 02:05:44 UTC,Jd,So... sorry to be a spoilsport but if <p>(1) The aforementioned class was the one in which you learned the most \nAND\n(2) Your teacher told you to read Howard Zinn if you wanted to learn something \nIMPLYING\n(2') Your teacher didn't think you were learning anything<p>Then mustn't we conclude (3) You didn't learn learn anything in your high school social studies classes<p>?,NaN
8,51669,51907,1189198809,2007-09-07 21:00:09 UTC,Jd,"I wrote two papers on Kant in uni, neither of which received an A. <p>My take is that intelligence follows a distribution curve for which graders at any decent college are only able to effectively evaluate the upper quartile, minus the top 1 or 2 percent. If you happen to be pushing the top 1 or 2 percent, you either have to lower yourself to the level of your graders or write your best thoughts and expect to receive less than the best grades for them. <p>If you follow the first track, you can expect the usual laurels for 'achievement.' If you follow the second, you can expect to increase your intelligence at a higher rate, which may (or may not) have some payback later on.<p>",NaN
9,40060,40090,1186500450,2007-08-07 15:27:30 UTC,Jd,"I think it could work for things which require a higher trust level or in which locality is a key factor. For example, suppose I have a hundred or so porn DVDs I inherited from my recently collapsed startup. Maybe I don't want to go through the hassle of sending each one out to a different user over ebay but if can give them to folks for $5 a pop

In [21]:
class HTMLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = HTMLStripper()
    s.feed(html)
    return s.get_data()

In [22]:
df = df[df['text'].notnull()]

In [23]:
df['text']  = df['text'].apply(html.unescape)
df['text'] = df['text'].apply(strip_tags)
df.head(10)

,parent,id,time,timestamp,by,text,score
0,39604,39629,1186355469,2007-08-05 23:11:09 UTC,Jd,"I'm in the 'Well duh' category too, but I still think Apple is over-rated. Too much enthusiasm for Jobs & co. will do no one any good.",NaN
1,39057,39157,1186178497,2007-08-03 22:01:37 UTC,Jd,perhaps clearer if I wrote 'profitability and the quality of the content',NaN
2,57130,57242,1190262050,2007-09-20 04:20:50 UTC,Jd,Does anything like this exist that you are aware of?,NaN
3,32291,32294,1183475440,2007-07-03 15:10:40 UTC,Jd,"Depends on where you go for college. The ivies I am familiar with separate the applications and financial aid process the issue of 'leniency' is not there (except when there are very large sums of money involved). As for private schools, I think you may be confused. Many of the leading private boarding schools are much better than almost any public schools in the nation. Counseling programs vary with the school. Schools which frequently send kids to the ivies (or ivy-equiv) tend to have lots of experience with the process. My public school had only a small amount. Some have none (and don't even know when one of their students has beaten the odds and made their way into such a school).",NaN
4,57812,57919,1190380082,2007-09-21 13:08:02 UTC,Jd,"Speculating on a causal principle implies that there is causality in the universe. Consequently, order and the possiblity of some degree of understanding are also implied, although the translation from speculation to empirical confirmation is not.",NaN
5,46389,46532,1188030337,2007-08-25 08:25:37 UTC,Jd,"This is a simple case of specialization of labor. While a hacker may be able to hear the voice of the customer, it probably makes sense to have some folks that do more hacking and some more customer hearing. If you have two co-founders, it makes sense to have one on each side.",NaN
6,39609,39627,1186355326,2007-08-05 23:08:46 UTC,Jd,"'Profit' is usually defined as pecuniary benefit. PG has stated elsewhere that he is not particularly concerned about this. In my charitable estimation, PG's ""Not a charity"" statement means that YC is not in the business of giving out financial handouts. It does not mean that PG's first or only motive is pecuniary benefit. We live in a culture defined by the worship of the Almighty dollar. If PG is at least hinting in a different direction, I'd say that's a good thing.",NaN
7,55442,55479,1189994744,2007-09-17 02:05:44 UTC,Jd,So... sorry to be a spoilsport but if (1) The aforementioned class was the one in which you learned the most \nAND\n(2) Your teacher told you to read Howard Zinn if you wanted to learn something \nIMPLYING\n(2') Your teacher didn't think you were learning anything Then mustn't we conclude (3) You didn't learn learn anything in your high school social studies classes ?,NaN
8,51669,51907,1189198809,2007-09-07 21:00:09 UTC,Jd,"I wrote two papers on Kant in uni, neither of which received an A. My take is that intelligence follows a distribution curve for which graders at any decent college are only able to effectively evaluate the upper quartile, minus the top 1 or 2 percent. If you happen to be pushing the top 1 or 2 percent, you either have to lower yourself to the level of your graders or write your best thoughts and expect to receive less than the best grades for them. If you follow the first track, you can expect the usual laurels for 'achievement.' If you follow the second, you can expect to increase your intelligence at a higher rate, which may (or may not) have some payback later on.",NaN
9,40060,40090,1186500450,2007-08-07 15:27:30 UTC,Jd,"I think it could work for things which require a higher trust level or in which locality is a key factor. For example, suppose I have a hundred or so porn DVDs I inherited from my recently collapsed startup. Maybe I don't want to go through the hassle of sending each one out to a different user over ebay but if can give them to folks for $5 a pop who happen to be around campus (or t

In [24]:
analyzer = SentimentIntensityAnalyzer()

In [25]:
def analyze(sentence):
    return analyzer.polarity_scores(sentence)

In [26]:
df['scores'] = df['text'].apply(analyze)
df[['neg', 'neu', 'pos', 'compound']] = df.scores.apply(pd.Series)
df = df.drop(columns="scores")
df.head(10)

,parent,id,time,timestamp,by,text,score,neg,neu,pos,compound
0,39604,39629,1186355469,2007-08-05 23:11:09 UTC,Jd,"I'm in the 'Well duh' category too, but I still think Apple is over-rated. Too much enthusiasm for Jobs & co. will do no one any good.",NaN,0.085,0.635,0.280,0.7543
1,39057,39157,1186178497,2007-08-03 22:01:37 UTC,Jd,perhaps clearer if I wrote 'profitability and the quality of the content',NaN,0.000,0.826,0.174,0.2732
2,57130,57242,1190262050,2007-09-20 04:20:50 UTC,Jd,Does anything like this exist that you are aware of?,NaN,0.000,0.783,0.217,0.3612
3,32291,32294,1183475440,2007-07-03 15:10:40 UTC,Jd,"Depends on where you go for college. The ivies I am familiar with separate the applications and financial aid process the issue of 'leniency' is not there (except when there are very large sums of money involved). As for private schools, I think you may be confused. Many of the leading private boarding schools are much better than almost any public schools in the nation. Counseling programs vary with the school. Schools which frequently send kids to the ivies (or ivy-equiv) tend to have lots of experience with the process. My public school had only a small amount. Some have none (and don't even know when one of their students has beaten the odds and made their way into such a school).",NaN,0.041,0.935,0.024,-0.2960
4,57812,57919,1190380082,2007-09-21 13:08:02 UTC,Jd,"Speculating on a causal principle implies that there is causality in the universe. Consequently, order and the possiblity of some degree of understanding are also implied, although the translation from speculation to empirical confirmation is not.",NaN,0.000,1.000,0.000,0.0000
5,46389,46532,1188030337,2007-08-25 08:25:37 UTC,Jd,"This is a simple case of specialization of labor. While a hacker may be able to hear the voice of the customer, it probably makes sense to have some folks that do more hacking and some more customer hearing. If you have two co-founders, it makes sense to have one on each side.",NaN,0.000,1.000,0.000,0.0000
6,39609,39627,1186355326,2007-08-05 23:08:46 UTC,Jd,"'Profit' is usually defined as pecuniary benefit. PG has stated elsewhere that he is not particularly concerned about this. In my charitable estimation, PG's ""Not a charity"" statement means that YC is not in the business of giving out financial handouts. It does not mean that PG's first or only motive is pecuniary benefit. We live in a culture defined by the worship of the Almighty dollar. If PG is at least hinting in a different direction, I'd say that's a good thing.",NaN,0.026,0.795,0.179,0.9164
7,55442,55479,1189994744,2007-09-17 02:05:44 UTC,Jd,So... sorry to be a spoilsport but if (1) The aforementioned class was the one in which you learned the most \nAND\n(2) Your teacher told you to read Howard Zinn if you wanted to learn something \nIMPLYING\n(2') Your teacher didn't think you were learning anything Then mustn't we conclude (3) You didn't learn learn anything in your high school social studies classes ?,NaN,0.018,0.982,0.000,-0.0387
8,51669,51907,1189198809,2007-09-07 21:00:09 UTC,Jd,"I wrote two papers on Kant in uni, neither of which received an A. My take is that intelligence follows a distribution curve for which graders at any decent college are only able to effectively evaluate the upper quartile, minus the top 1 or 2 percent. If you happen to be pushing the top 1 or 2 percent, you either have to lower yourself to the level of your graders or write your best thoughts and expect to receive less than the best grades for them. If you follow the first track, you can expect the usual laurels for 'achievement.' If you follow the second, you can expect to increase your intelligence at a higher rate, which may (or may not) have some payback later on.",NaN,0.016,0.811,0.173,0.9648
9,40060,40090,1186500450,2007-08-07 15:27:30 UTC,Jd,"I think it could work for things which require a higher trust level or in which locality is a key factor. For example, suppose I have a h

In [27]:
for text in df.sort_values(by='neg', ascending=False)['text'].head(20):
    print(text, '\n')

No 

SPAM 

jealousy. 

dupe 

No. 

Heh heh heh.  

Desperation? 

No. 

Seriously. 

No. 

desperation 

spam 

depressing. 

:( 

scary. 

Fools.  

No. 

dupe 

dupe 

No. 



In [28]:
df.to_csv('first_cleaning_with_scores.csv', index=False)

In [29]:
df.head(50)

,parent,id,time,timestamp,by,text,score,neg,neu,pos,compound
0,39604,39629,1186355469,2007-08-05 23:11:09 UTC,Jd,"I'm in the 'Well duh' category too, but I still think Apple is over-rated. Too much enthusiasm for Jobs & co. will do no one any good.",NaN,0.085,0.635,0.280,0.7543
1,39057,39157,1186178497,2007-08-03 22:01:37 UTC,Jd,perhaps clearer if I wrote 'profitability and the quality of the content',NaN,0.000,0.826,0.174,0.2732
2,57130,57242,1190262050,2007-09-20 04:20:50 UTC,Jd,Does anything like this exist that you are aware of?,NaN,0.000,0.783,0.217,0.3612
3,32291,32294,1183475440,2007-07-03 15:10:40 UTC,Jd,"Depends on where you go for college. The ivies I am familiar with separate the applications and financial aid process the issue of 'leniency' is not there (except when there are very large sums of money involved). As for private schools, I think you may be confused. Many of the leading private boarding schools are much better than almost any public schools in the nation. Counseling programs vary with the school. Schools which frequently send kids to the ivies (or ivy-equiv) tend to have lots of experience with the process. My public school had only a small amount. Some have none (and don't even know when one of their students has beaten the odds and made their way into such a school).",NaN,0.041,0.935,0.024,-0.2960
4,57812,57919,1190380082,2007-09-21 13:08:02 UTC,Jd,"Speculating on a causal principle implies that there is causality in the universe. Consequently, order and the possiblity of some degree of understanding are also implied, although the translation from speculation to empirical confirmation is not.",NaN,0.000,1.000,0.000,0.0000
5,46389,46532,1188030337,2007-08-25 08:25:37 UTC,Jd,"This is a simple case of specialization of labor. While a hacker may be able to hear the voice of the customer, it probably makes sense to have some folks that do more hacking and some more customer hearing. If you have two co-founders, it makes sense to have one on each side.",NaN,0.000,1.000,0.000,0.0000
6,39609,39627,1186355326,2007-08-05 23:08:46 UTC,Jd,"'Profit' is usually defined as pecuniary benefit. PG has stated elsewhere that he is not particularly concerned about this. In my charitable estimation, PG's ""Not a charity"" statement means that YC is not in the business of giving out financial handouts. It does not mean that PG's first or only motive is pecuniary benefit. We live in a culture defined by the worship of the Almighty dollar. If PG is at least hinting in a different direction, I'd say that's a good thing.",NaN,0.026,0.795,0.179,0.9164
7,55442,55479,1189994744,2007-09-17 02:05:44 UTC,Jd,So... sorry to be a spoilsport but if (1) The aforementioned class was the one in which you learned the most \nAND\n(2) Your teacher told you to read Howard Zinn if you wanted to learn something \nIMPLYING\n(2') Your teacher didn't think you were learning anything Then mustn't we conclude (3) You didn't learn learn anything in your high school social studies classes ?,NaN,0.018,0.982,0.000,-0.0387
8,51669,51907,1189198809,2007-09-07 21:00:09 UTC,Jd,"I wrote two papers on Kant in uni, neither of which received an A. My take is that intelligence follows a distribution curve for which graders at any decent college are only able to effectively evaluate the upper quartile, minus the top 1 or 2 percent. If you happen to be pushing the top 1 or 2 percent, you either have to lower yourself to the level of your graders or write your best thoughts and expect to receive less than the best grades for them. If you follow the first track, you can expect the usual laurels for 'achievement.' If you follow the second, you can expect to increase your intelligence at a higher rate, which may (or may not) have some payback later on.",NaN,0.016,0.811,0.173,0.9648
9,40060,40090,1186500450,2007-08-07 15:27:30 UTC,Jd,"I think it could work for things which require a higher trust level or in which locality is a key factor. For example, suppose I have a h